In [1]:
from hepdata_lib import RootFileReader
from hepdata_lib import Table
from hepdata_lib import Variable, Uncertainty
from hepdata_lib import Submission
import math
def round_pair(array, n):
    newx = []
    for x in array:
        newx.append((round(x[0],n), round(x[1],n)))
    return newx
"""
def round_list(array,n):
    m = len(array)
    newdata = []
    for i in range(m):
        x = data[i]%1
        ndata = math.ceil(n-math.log10(y))
        newdata.append(round(data[i], ndata))
    return newdata
"""


def round_hist(data, stat, n):
    m = len(data)
    newdata = []
    newstat = []
    for i in range(m):
        x = stat[i]%1
        y = data[i]%1
        ndata = math.ceil(n-math.log10(y))
        n0 = math.ceil(n-math.log10(x))
        newdata.append(round(data[i], n0))
        newstat.append(round(stat[i], n0))
    return newdata, newstat

def round_hist_syst(data, stat, syst, n):
    m = len(data)
    newdata = []
    newsyst = []
    newstat = []
    for i in range(m):
        d = data[i]%1
        ndata = math.ceil(n-math.log10(d))
        x = stat[i]%1
        y = syst[i]%1
        n0 = math.ceil(n-math.log10(min(x,y)))
        newdata.append(round(data[i], n0))
        newstat.append(round(stat[i], n0))
        newsyst.append(round(syst[i], n0))
    return newdata, newstat, newsyst

Welcome to JupyROOT 6.22/02


In [2]:
reader = RootFileReader("Results/jet_shapes.root")
hjs_incl = reader.read_hist_1d('dr_data_incl_js_allpt_0_0')
hjs_b = reader.read_hist_1d('dr_data_bjtc_js_allpt_0_0')
hratio = reader.read_hist_1d('division_dr_data_bjtc_js_allpt_0_0')
hjs_incl_syst = reader.read_hist_1d('dr_data_incl_allpt_syst_err_0_0')
hjs_b_syst = reader.read_hist_1d('dr_data_bjtc_allpt_syt_err_0_0')
hratio_syst = reader.read_hist_1d('division_dr_data_bjtc_allpt_syt_err_0_0')

reader2 = RootFileReader("Results/particle_yield.root")
hpy_incl = reader2.read_hist_1d('js_incl_tot')
hpy_b = reader2.read_hist_1d('js_b_tot')
hdiff = reader2.read_hist_1d('diff_tot')
hpy_incl_syst = reader2.read_hist_1d('js_incl_err_tot')
hpy_b_syst = reader2.read_hist_1d('js_b_err_tot')
hdiff_syst = reader2.read_hist_1d('diff_err_tot')

hjs_b_pt = []
hjs_incl_pt=[]
hpy_b_pt = []
hpy_incl_pt=[]
for i in range(6):
    hjs_b_pt.append(reader.read_hist_1d('dr_signal_bjtc_jetShape_step3_{0}_0'.format(i)))
    hjs_incl_pt.append(reader.read_hist_1d('dr_signal_dijtc_jetShape_step3_{0}_0'.format(i)))
for i in range(5):
    hpy_b_pt.append(reader.read_hist_1d('dr_signal_bjtc_jetShape_step3_{0}_0'.format(i)))
    hpy_incl_pt.append(reader.read_hist_1d('dr_signal_dijtc_jetShape_step3_{0}_0'.format(i)))

In [3]:
## load data
# x-axis : dr 
ndigi = 1
dr = Variable("$\Delta r$", is_independent = True)
dr.values = round_pair(hjs_incl["x_edges"][:11],ndigi)

# y-axis : jet shapes

ptlabel = ["1<p_T^{\\text{trk}}<2",
           "2<p_T^{\\text{trk}}<3",
           "3<p_T^{\\text{trk}}<4",
           "4<p_T^{\\text{trk}}<8",
           "8<p_T^{\\text{trk}}<12",
           "p_T^{\\text{trk}} > 12",
          ]
js_incl_pt= []
js_incl_pt_err=[]
js_b_pt= []
js_b_pt_err=[]
for i in range(6):
    js_incl_pt.append(
        Variable("$\\rho (\Delta r)_{\\text{incl.}}$",
                 is_independent = False, is_binned=False, units=""))
    js_incl_pt[i].add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
    js_incl_pt[i].add_qualifier("RE", "pp-->Charged X", "")
    js_incl_pt[i].add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
    js_incl_pt[i].add_qualifier("Track Rapidity", "$|\eta^{\\text{trk}}|<2.4$", "")
    js_incl_pt[i].add_qualifier("Track Kinematics", 
                                "${ptlabel}$".format(ptlabel=ptlabel[i]), "")
    js_incl_pt_err = Uncertainty("Statistical Uncert", is_symmetric = True)
    syst_err = Uncertainty("Systematic Uncert", is_symmetric = True)
    js_incl_pt[i].values, js_incl_pt_err.values = round_hist(hjs_incl_pt[i]["y"][:11],hjs_incl_pt[i]['dy'][:11],ndigi)
    js_incl_pt[i].add_uncertainty(js_incl_pt_err)
    
    js_b_pt.append(
        Variable("$\\rho(\Delta r)_{\\text{b}}$", is_independent = False, is_binned=False, units=""))
    js_b_pt[i].add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
    js_b_pt[i].add_qualifier("RE", "pp-->Charged X", "")
    js_b_pt[i].add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
    js_b_pt[i].add_qualifier("Track Rapidity", "$|\eta^{\\text{trk}}|<2.4$", "")
    js_b_pt[i].add_qualifier("Track Kinematics", 
                             "${ptlabel}$".format(ptlabel=ptlabel[i]),"")
    js_b_pt_err=Uncertainty("Statistical Uncert", is_symmetric = True)
    js_b_pt[i].values, js_b_pt_err.values = round_hist(hjs_b_pt[i]["y"][:11],hjs_b_pt[i]['dy'][:11],ndigi)
    js_b_pt[i].add_uncertainty(js_b_pt_err)
    
    
js_incl = Variable("$\\rho(\Delta r)_{\\text{incl.}}$", is_independent = False, is_binned=False, units="")
js_incl.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
js_incl.add_qualifier("RE", "pp-->Charged X", "")
js_incl.add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
js_incl.add_qualifier("Track Kinematics", "$p_{\\text{T}} > 1 \\text{ GeV}, |\eta| < 2.4$","")

js_incl_err = Uncertainty("Statistical Uncert", is_symmetric = True)
js_incl_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
js_incl.values, js_incl_err.values, js_incl_syst.values = round_hist_syst(
    hjs_incl['y'][:11],hjs_incl['dy'][:11],hjs_incl_syst['dy'][:11],ndigi)
js_incl.add_uncertainty(js_incl_syst)
js_incl.add_uncertainty(js_incl_err)



js_b = Variable("$\\rho(\Delta r)_{\\text{b}}$", is_independent = False, is_binned=False, units="")
js_b.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
js_b.add_qualifier("RE", "pp-->Charged X", "")
js_b.add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
js_b.add_qualifier("Track Kinematics", "$p_{\\text{T}} > 1 \\text{ GeV}, |\eta| < 2.4$","")


js_b_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
js_b_err = Uncertainty("Statistical Uncert", is_symmetric = True)

js_b.values, js_b_err.values, js_b_syst.values = round_hist_syst(
    hjs_b["y"][:11],hjs_b['dy'][:11],hjs_b_syst['dy'][:11],ndigi)

js_b.add_uncertainty(js_b_syst)
js_b.add_uncertainty(js_b_err)

ratio = Variable("$\\rho(\Delta r)_{\\text{b}}/\\rho(\Delta r)_{\\text{incl.}}$", is_independent = False, is_binned=False, units="")
ratio.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
ratio.add_qualifier("RE", "pp-->Charged X", "")
ratio.add_qualifier("round_listJet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
ratio.add_qualifier("Track Kinematics", "$p_{\\text{T}} > 1 \\text{ GeV}, |\eta| < 2.4$","")

ratio_syst = Uncertainty("Systematic Uncer", is_symmetric = True)
ratio_err = Uncertainty("Statistical Uncert", is_symmetric = True)
ratio.values,ratio_err.values,ratio_syst.values = round_hist_syst(
    hratio["y"][:11],hratio['dy'][:11],hratio_syst['dy'][:11],ndigi)

ratio.add_uncertainty(ratio_syst)
ratio.add_uncertainty(ratio_err)

# y-axis : particle yield 
pt = [1,2,3,4,8, 12]
py_incl_pt= []
py_b_pt= []
py_b_pt_err=[]
for i in range(5):
    py_incl_pt.append(
        Variable("$Y(\Delta r)_{\\text{incl.}}$", is_independent = False, is_binned=False, units=""))
    py_incl_pt[i].add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
    py_incl_pt[i].add_qualifier("RE", "pp-->Charged X", "")
    py_incl_pt[i].add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
    py_incl_pt[i].add_qualifier("Track Rapidity", "$|\eta^{\\text{trk}}|<2.4$", "")
    py_incl_pt[i].add_qualifier("Track Kinematics", 
                                "${ptlabel}$".format(ptlabel=ptlabel[i]), "")
    py_incl_pt_err=Uncertainty("Statistical Uncert", is_symmetric = True)
    py_incl_pt[i].values, py_incl_pt_err.values= round_hist(
        hpy_incl_pt[i]['y'][:11], hpy_incl_pt[i]['dy'][:11],ndigi)
    py_incl_pt[i].add_uncertainty(py_incl_pt_err)
    
    py_b_pt.append(
        Variable("$Y(\Delta r)_{\\text{b}}$", is_independent = False, is_binned=False, units=""))
    py_b_pt[i].add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
    py_b_pt[i].add_qualifier("RE", "pp-->Charged X", "")
    py_b_pt[i].add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
    py_b_pt[i].add_qualifier("Track Rapidity", "$|\eta^{\\text{trk}}|<2.4$", "")
    py_b_pt[i].add_qualifier("Track Kinematics", 
                                "${ptlabel}$".format(ptlabel=ptlabel[i]), "")
    
    py_b_pt_err = Uncertainty("Statistical Uncert", is_symmetric = True)
    
    py_b_pt[i].values, py_b_pt_err.values = round_hist(
        hpy_incl_pt[i]['y'][:11], hpy_incl_pt[i]['dy'][:11],ndigi)
    py_b_pt[i].add_uncertainty(py_b_pt_err)

py_incl = Variable("$Y(\Delta r)_{\\text{incl.}}$", is_independent = False, is_binned=False, units="")
py_incl.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
py_incl.add_qualifier("RE", "pp-->Charged X", "")
py_incl.add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
py_incl.add_qualifier("Track Kinematics", "$1 < p_{\\text{T}} < 12 \\text{ GeV}, |\eta| < 2.4$","")

py_incl_err = Uncertainty("Statistical Uncert", is_symmetric = True)
py_incl_syst = Uncertainty("Systematic Uncert", is_symmetric = True)

py_incl.values, py_incl_err.values,py_incl_syst.values = round_hist_syst(
    hpy_incl["y"][:11], hpy_incl['dy'][:11],hpy_incl_syst['dy'][:11],ndigi)
py_incl.add_uncertainty(py_incl_err)
py_incl.add_uncertainty(py_incl_syst)

py_b = Variable("$Y(\Delta r)_{\\text{b}}$", is_independent = False, is_binned=False, units="")
py_b.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
py_b.add_qualifier("RE", "pp-->Charged X", "")
py_b.add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
py_b.add_qualifier("Track Kinematics", "$1 < p_{\\text{T}} < 12 \\text{ GeV}, |\eta| < 2.4$","")

py_b_syst = Uncertainty("Systematic Uncert", is_symmetric = True)
py_b_err = Uncertainty("Statistical Uncert", is_symmetric = True)
py_b.values,py_b_err.values, py_b_syst.values = round_hist_syst(
    hpy_b['y'][:11], hpy_b['dy'][:11], hpy_b_syst['dy'][:11],ndigi)
py_b.add_uncertainty(py_b_syst)
py_b.add_uncertainty(py_b_err)

diff = Variable("$Y(\Delta r)_{\\text{b}}-Y(\Delta r)_{\\text{incl.}}$", is_independent = False, is_binned=False, units="")
diff.add_qualifier("SQRT(S)/NUCLEON", 5020, "GeV")
diff.add_qualifier("RE", "pp-->Charged X", "")
diff.add_qualifier("Jet Kinematics", "$p_{\\text{T}} > 120 \\text{ GeV}, |\eta| < 1.6$", "")
diff.add_qualifier("Track Kinematics", "$1 < p_{\\text{T}} < 12 \\text{ GeV}, |\eta| < 2.4$","")

diff_syst = Uncertainty("Systematic Uncer", is_symmetric = True)
diff_err = Uncertainty("Statistical Uncert", is_symmetric = True)

diff.values, diff_err.values, diff_syst.values = round_hist_syst(
    hdiff["y"][:11],hdiff["dy"][:11], hdiff_syst['dy'][:11],ndigi)
diff.add_uncertainty(diff_syst)
diff.add_uncertainty(diff_err)

In [4]:
submission = Submission()
table1l = Table("Figure 1 (left)")
table1l.description = "The charged particle yield distribution $Y(\Delta r)$ of inclusive jets with $p_T > 120$ GeV and $1< p^{\\text{trk}}_T < 12$ GeV are presented as functions of $\Delta r$ for differential $p_{\\text{T}}^{\\text{trk}}$ bin."
table1l.keywords["observables"]=["Charged Particle Yield"]
table1l.keywords["reactions"]=["proton proton --> jets"]
table1l.keywords["cmenergies"]=["5020"]
table1l.add_variable(dr)
for h in py_incl_pt:
    table1l.add_variable(h)
submission.add_table(table1l)
    
table1r = Table("Figure 1 (right)")
table1r.description = "The charged particle yield distribution $Y(\Delta r)$ of b jets with $p_T > 120$ GeV and $1< p^{\\text{trk}}_T < 12$ GeV are presented as functions of $\Delta r$ for differential $p_{\\text{T}}^{\\text{trk}}$ bin."
table1r.keywords["observables"]=["Charged Particle Yield"]
table1r.keywords["reactions"]=["proton proton --> jets"]
table1r.keywords["cmenergies"]=["5020"]
table1r.add_variable(dr)
for h in py_b_pt:
    table1r.add_variable(h)
submission.add_table(table1r)

In [5]:
table2l = Table("Figure 2 (left)")
table2l.description = "Charged particle yield distributions $Y(\Delta r)$ of inclusive jets with $1 < p_{\\text{T}}^{\\text{trk}} < 12$ GeV are presented as functions of $\Delta r$.\
Inclusive jets with $p_T > 120$ GeV and charged particles with $1 < p^{\\text{trk}}_{\\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_{\\text{T}}^{\\text{trk}}$ bins."
table2l.keywords["cmenergies"]=["5020"]
table2l.keywords["reactions"]=["proton proton --> jets"]
table2l.keywords["observables"]=["Charged Particle Yield"]
table2l.add_variable(dr)
table2l.add_variable(py_incl)
submission.add_table(table2l)

table2m = Table("Figure 2 (middle)")
table2m.description = "Charged particle yield distributions $Y(\Delta r)$ of b jets with $1 < p_{\\text{T}}^{\\text{trk}} < 12$ GeV are presented as functions of $\Delta r$.\
b jets with $p_T > 120$ GeV and charged particles with $1 < p^{\\text{trk}}_{\\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_{\\text{T}}^{\\text{trk}}$ bins."
table2m.keywords["cmenergies"]=["5020"]
table2m.keywords["reactions"]=["proton proton --> jets"]
table2m.keywords["observables"]=["Charged Particle Yield"]
table2m.add_variable(dr)
table2m.add_variable(py_b)
submission.add_table(table2m)

table2r = Table("Figure 2 (right)")
table2r.description = "Charged particle yield differences of b to inclusive jets for $1 < p_{\\text{T}}^{\\text{trk}} < 12$ GeV are presented as functions of $\Delta r$.\
b jets with $p_T > 120$ GeV and charged particles with $1 < p^{\\text{trk}}_{\\text{T}} < 12$ GeV are used to construct the distributions as functions of $\Delta r$ differential $p_{\\text{T}}^{\\text{trk}}$ bins."
table2r.keywords["cmenergies"]=["5020"]
table2r.keywords["reactions"]=["proton proton --> jets"]
table2r.keywords["observables"]=["Charged Particle Yield"]
table2r.add_variable(dr)
table2r.add_variable(diff)
submission.add_table(table2r)

In [6]:
table3l = Table("Figure 3 (left)")
table3l.description = "The jet shapes $\\rho(\Delta r)$ of inclusive jets with $p_{\\text{T}} > 120$ GeV and $p^{trk}_T > 1$ GeV are presented as functions of $\Delta r$ for differential $p_{\\text{T}}^{\\text{trk}}$ bin."
table3l.keywords["cmenergies"]=["5020"]
table3l.keywords["reactions"]=["proton proton --> jets"]
table3l.keywords["observables"]=["Jet Shapes"]
table3l.add_variable(dr)
for h in js_incl_pt:
    table3l.add_variable(h)
submission.add_table(table3l)
    
table3r = Table("Figure 3 (right)")
table3r.description = "The jet shape distributions $\\rho(\Delta r)$ of b jets with $p_{\\text{T}} > 120$ GeV and $1< p^{\\text{trk}}_{\\text{T}} < 12$ GeV are presented as functions of $\Delta r$ for differential $p_{\\text{T}}^{\\text{trk}}$ bin."
table3r.keywords["cmenergies"]=["5020"]
table3r.keywords["reactions"]=["proton proton --> jets"]
table3r.keywords["observables"]=["Jet Shapes"]
table3r.add_variable(dr)
for h in js_b_pt:
    table3r.add_variable(h)
submission.add_table(table3r)

In [7]:
table4l = Table("Figure 4 (left)")
table4l.description = "The jet shapes $\\rho (\Delta r)$ of inclusive jets with $p_{\\text{T}} > 120$ GeV and $p^{\\text{trk}}_{\\text{T}} > 1$ GeV are presented as functions of $\Delta r."
table4l.keywords["cmenergies"]=["5020"]
table4l.keywords["reactions"]=["proton proton --> jets"]
table4l.keywords["observables"]=["Jet Shapes"]
table4l.add_variable(dr)
table4l.add_variable(js_incl)
submission.add_table(table4l)

table4m = Table("Figure 4 (middle)")
table4m.description = "The jet shapes $\\rho (\Delta r)$ of b jets with $p_{\\text{T}} > 120$ GeV and $p^{\\text{trk}}_{\\text{T}} > 1$ GeV are presented as functions of $\Delta r."
table4m.keywords["cmenergies"]=["5020"]
table4m.keywords["reactions"]=["proton proton --> jets"]
table4m.keywords["observables"]=["Jet Shapes"]
table4m.add_variable(dr)
table4m.add_variable(js_b)
submission.add_table(table4m)

table4r = Table("Figure 4 (right)")
table4r.description = "The b-to-inclusive jet shape ratios for $p_{\\text{T}} > 120$ GeV and $p^{\\text{trk}}_{\\text{T}} > 1$ GeV are presented as functions of $\Delta r."
table4r.keywords["cmenergies"]=["5020"]
table4r.keywords["reactions"]=["proton proton --> jets"]
table4r.keywords["observables"]=["Jet Shapes"]
table4r.add_variable(dr)
table4r.add_variable(ratio)
submission.add_table(table4r)



In [8]:
submission.create_files("hin_18_020_hepdata")

In [9]:
!head hin_18_020_hepdata/figure_1_(right).yaml

/bin/bash: -c: line 0: syntax error near unexpected token `('
/bin/bash: -c: line 0: `head hin_18_020_hepdata/figure_1_(right).yaml'
